In [1]:
from helper_functions import *
import plotly.graph_objects as go
from plotly.colors import sample_colorscale, hex_to_rgb

In [2]:
#file_path = '../output/clusters/paper/strain_rate_2.6e+04_safety_factor_4.0e-01_time1.0e-04_contact10/data.h5'
file_path = '../output/clusters/paper/strain_rate_5.0e+03_safety_factor_4.0e-01_time1.0e-04_contact10/data.h5'
data = readH5Data(file_path)
print(data.keys())

dict_keys(['times', 'contact_overshoot', 'econ', 'edis', 'ekin', 'epot', 'erev', 'initial_unstable_state', 'nb_fragments', 'num_contact_overshoot', 'num_initial_unstable_state', 'num_softening_overshoot', 'softening_overshoot', 'time', 'total_energy', 'work', 'fragment_mass', 'fragment_velocity'])


### Energies partitioning

In [12]:
colorscale = 'Spectral_r'
uniform_sampling = False
if uniform_sampling:
    n_colors = 5
    samples = [i/(n_colors-1) for i in range(n_colors)]
    colors = sample_colorscale(colorscale, samples)
else:
    samples = [0., 0.1, 0.25, 0.35, 0.6]
    colors = sample_colorscale(colorscale, samples)

fillcolors = [c.replace('rgb(', 'rgba(').replace(')', ',0.7)') for c in colors]

In [13]:
# Plot energy vs time
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['time'], y=data['ekin']-data['ekin'][0], mode='lines', name=r"$\mathcal{K}$", line=dict(color=colors[0])))
fig.add_trace(go.Scatter(x=data['time'], y=data['epot']-data['epot'][0], mode='lines', name=r"$\mathcal{U}$", line=dict(color=colors[1])))
fig.add_trace(go.Scatter(x=data['time'], y=data['erev']-data['erev'][0], mode='lines', name=r"$\mathcal{R}$", line=dict(color=colors[2])))
fig.add_trace(go.Scatter(x=data['time'], y=data['econ']-data['econ'][0], mode='lines', name=r"$\mathcal{C}$", line=dict(color=colors[3])))
fig.add_trace(go.Scatter(x=data['time'], y=data['edis']-data['edis'][0], mode='lines', name=r"$\mathcal{G}$", line=dict(color='grey')))

#fig.add_trace(go.Scatter(x=data['time'], y=data['work']-data['work'][0], mode='lines', name=r"\mathcal{W}_\text{ext}"))
fig.add_trace(go.Scatter(x=data['time'], y=data['total_energy']-data['total_energy'][0], mode='lines', name='Total Energy',line=dict(color='black', width=2, dash='dot')))

fig.update_layout(
    title='Energy vs Time',
    xaxis_title='Time (s)',
    yaxis_title=r"$\Delta\mathcal{E}$",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        tickfont=dict(size=10),
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        tickfont=dict(size=10),  
    ),
    font=dict(
        family="Latin-Modern",
        size=12,
        color="Black"
    ),
    legend=dict(
        x=0.97,              # position from the left (0 to 1)
        y=0.35,              # position from the bottom (0 to 1)
        bgcolor='rgba(255, 255, 255, 0.8)',  # white with 80% opacity
        bordercolor='black',
        borderwidth=1,
        orientation='h',   
        xanchor='right',
        yanchor='top',
    ),
    width=400, 
    height=400,
    showlegend=False,
    template="plotly_white")

fig.show()

In [5]:
inj_energy_final = data['work'][-1]+data['ekin'][0]+data['epot'][0]
ekin = (data['ekin']) / inj_energy_final
epot = (data['epot']+data['ekin']) / inj_energy_final
erev = (data['erev']+data['epot']+data['ekin']) / inj_energy_final
econ = (data['econ']+data['erev']+data['epot']+data['ekin']) / inj_energy_final
edis = (data['edis']+data['econ']+data['erev']+data['epot']+data['ekin']) / inj_energy_final

einj = (data['work']+data['ekin'][0]+data['epot'][0]) / inj_energy_final


fig = go.Figure()

fig.add_trace(go.Scatter(x=data['time'], y=ekin, mode='lines', name=r"$\mathcal{K}$", fill='tozeroy', line=dict(color=colors[0]), fillcolor=fillcolors[0]))
fig.add_trace(go.Scatter(x=data['time'], y=epot, mode='lines', name=r"$\mathcal{U}$", fill='tonexty', line=dict(color=colors[1]), fillcolor=fillcolors[1]))
fig.add_trace(go.Scatter(x=data['time'], y=erev, mode='lines', name=r"$\mathcal{R}$", fill='tonexty', line=dict(color=colors[2]), fillcolor=fillcolors[2]))
fig.add_trace(go.Scatter(x=data['time'], y=econ, mode='lines', name=r"$\mathcal{C}$", fill='tonexty', line=dict(color=colors[3]), fillcolor=fillcolors[3]))
fig.add_trace(go.Scatter(x=data['time'], y=edis, mode='lines', name=r"$\mathcal{G}$", fill='tonexty', line=dict(color='lightgrey')))#, fillpattern=dict(shape='x')))
fig.add_trace(go.Scatter(x=data['time'], y=einj, mode='lines', name=r"$\mathcal{W}_\text{ext} + \mathcal{E}_0$", line=dict(dash='dot', color='black', width=2)))


fig.update_layout(
    title='Energy vs Time',
    xaxis_title='Time (s)',
    yaxis_title=r"$\mathcal{E}/\mathcal{E}_\text{inj}$",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        showexponent='last',
        tickfont=dict(size=12),
        #range=[0, 1e-4],
        dtick=2e-5,
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        tickfont=dict(size=12),
        #range=[0, 2],  
    ),
    font=dict(
        family="Latin-Modern",
        size=12,
        color="Black"
    ),
    legend=dict(
        x=0.97,              # position from the left (0 to 1)
        y=0.35,              # position from the bottom (0 to 1)
        bgcolor='rgba(255, 255, 255, 0.8)',  # white with 80% opacity
        bordercolor='black',
        borderwidth=1,
        orientation='h',   
        xanchor='right',
        yanchor='top',
    ),
    width=400, 
    height=400,
    showlegend=False,
    template="plotly_white")

fig.show()
#fig.write_image("../output/plots/paper/energy_vs_time_unstable_t04.pdf", width=400, height=400)

In [6]:
inj_energy_final = data['work'][-1]+data['ekin'][0]+data['epot'][0]
ekin = (data['ekin']) / inj_energy_final
epot = (data['epot']+data['ekin']) / inj_energy_final
erev = (data['erev']+data['epot']+data['ekin']) / inj_energy_final
econ = (data['econ']+data['erev']+data['epot']+data['ekin']) / inj_energy_final
edis = (data['edis']+data['econ']+data['erev']+data['epot']+data['ekin']) / inj_energy_final

einj = (data['work']+data['ekin'][0]+data['epot'][0]) / inj_energy_final


fig = go.Figure()

fig.add_trace(go.Scatter(x=data['time'], y=ekin, mode='lines', name=r"$\mathcal{K}$", fill='tozeroy', line=dict(color=colors[0], width=0), fillcolor=fillcolors[0]))
fig.add_trace(go.Scatter(x=data['time'], y=epot, mode='lines', name=r"$\mathcal{U}$", fill='tonexty', line=dict(color=colors[1], width=0), fillcolor=fillcolors[1]))
fig.add_trace(go.Scatter(x=data['time'], y=erev, mode='lines', name=r"$\mathcal{R}$", fill='tonexty', line=dict(color=colors[2], width=0), fillcolor=fillcolors[2]))
fig.add_trace(go.Scatter(x=data['time'], y=econ, mode='lines', name=r"$\mathcal{C}$", fill='tonexty', line=dict(color=colors[3], width=0), fillcolor=fillcolors[3]))
fig.add_trace(go.Scatter(x=data['time'], y=edis, mode='lines', name=r"$\mathcal{G}$", fill='tonexty', line=dict(color='lightgrey', width=0)))#, fillpattern=dict(shape='x')))
fig.add_trace(go.Scatter(x=data['time'], y=edis, mode='lines', name=r"$\mathcal{E}_\text{tot}$", line=dict(dash='solid', color='red', width=2)))#, fillpattern=dict(shape='x')))
fig.add_trace(go.Scatter(x=data['time'], y=einj, mode='lines', name=r"$\mathcal{W}_\text{ext} + \mathcal{E}_0$", line=dict(dash='dot', color='black', width=2)))


fig.update_layout(
    title='Energy vs Time',
    xaxis_title='Time (s)',
    yaxis_title=r"$\mathcal{E}/\mathcal{E}_\text{inj}$",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        showexponent='last',
        tickfont=dict(size=12),
        #range=[0, 1e-6],
        dtick=5e-7,
        side='top',
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        tickfont=dict(size=12),
        #range=[0, 1.1],  
    ),
    font=dict(
        family="Latin-Modern",
        size=12,
        color="Black"
    ),
    legend=dict(
        x=0.97,              # position from the left (0 to 1)
        y=1.05,              # position from the bottom (0 to 1)
        bgcolor='rgba(255, 255, 255, 0.8)',  # white with 80% opacity
        bordercolor='black',
        borderwidth=1,
        orientation='h',   
        xanchor='right',
        yanchor='top',
    ),
    width=400, 
    height=400,
    showlegend=False,
    template="plotly_white")

fig.show()
#fig.write_image("../output/plots/paper/energy_vs_time_unstable_t04_zoom_v2.pdf", width=250, height=260)

### Number of fragments

In [7]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['time'], y=data['nb_fragments'], mode='lines', name='Number of Fragments', line=dict(color='black', width=2)))
fig.update_layout(
    title='Fragment count vs Time',
    xaxis_title='Time (s)',
    yaxis_title=r"$N_\text{frag}$",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        showexponent='last',
        tickfont=dict(size=12),
        #range=[0, 2e-5],
        #dtick=4e-4,
        # Position on top
        side='bottom',
    ),
    yaxis=dict(
        showgrid=True,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        tickfont=dict(size=12),  
        #range=[0, 250],
    ),
    font=dict(
        family="Latin-Modern",
        size=12,
        color="Black"
    ),
    width=400, 
    height=400,
    showlegend=False,
    template="plotly_white")

fig.show()
#fig.write_image("../output/plots/paper/fragment_count_vs_time_unstable_t04_2_shorttime.pdf", width=400, height=400)#

### Instability types

In [8]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['time'], y=data['num_contact_overshoot'], mode='lines', name='Contact', line=dict(color='red', width=2)))
fig.add_trace(go.Scatter(x=data['time'], y=data['num_softening_overshoot'], mode='lines', name='Softening', line=dict(color='black', width=2)))
#fig.add_trace(go.Scatter(x=data['time'], y=data['num_initial_unstable_state'], mode='lines', name='Initial Unstable State', line=dict(color='blue', width=2)))
fig.update_layout(
    title='Instability type vs Time',
    xaxis_title='Time (s)',
    yaxis_title=r"$N_\text{q}$",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        showexponent='last',
        dtick=2e-5,
        tickfont=dict(size=12),
        range=[0, 1e-4],
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        tickfont=dict(size=12),
        #type='log',  
        range=[0, 8e4],
    ),
    font=dict(
        family="Latin-Modern",
        size=12,
        color="Black"
    ),
    legend=dict(
        x=0.9,              # position from the left (0 to 1)
        y=0.58,              # position from the bottom (0 to 1)
        bgcolor='rgba(255, 255, 255, 0.5)',  # white with 80% opacity
        bordercolor='black',
        borderwidth=1,
        orientation='v',   
        xanchor='right',
        yanchor='top',
    ),
    width=400, 
    height=400,
    showlegend=True,
    template="plotly_white")
fig.show()
#fig.write_image("../output/plots/paper/overshoot_vs_time_unstable_t04.pdf", width=400, height=280)

In [9]:
# Smoothen ekin and plot its derivative
import numpy as np
def smooth(data, window_size=5):
    return np.convolve(data, np.ones(window_size)/window_size, mode='valid')

ekin_smooth = smooth(data['ekin'], window_size=100) 
ekin_derivative = np.gradient(ekin_smooth, data['time'][1] - data['time'][0])

In [10]:
fig = go.Figure()
fig.add_trace(go.Scatter(x=data['time'], y=data['num_contact_overshoot'], mode='lines', name='Contact', line=dict(color='red', width=2), yaxis='y1'))
fig.add_trace(go.Scatter(x=data['time'][:len(ekin_smooth)], y=ekin_derivative, mode='lines', name=r"$\frac{d\mathcal{K}}{dt}$", line=dict(color="black"), yaxis='y2'))
#fig.add_trace(go.Scatter(x=data['time'], y=data['ekin'], mode='lines', name=r"$\mathcal{K}$", line=dict(color=colors[0]), yaxis='y2'))
#fig.add_trace(go.Scatter(x=data['time'], y=data['num_softening_overshoot'], mode='lines', name='Softening', line=dict(color='black', width=2)))
#fig.add_trace(go.Scatter(x=data['time'], y=data['num_initial_unstable_state'], mode='lines', name='Initial Unstable State', line=dict(color='blue', width=2)))
fig.update_layout(
    title='Instability type vs Time',
    xaxis_title='Time (s)',
    yaxis_title=r"$N_\text{q}$",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        showexponent='last',
        dtick=2e-5,
        tickfont=dict(size=12),
        range=[0, 0.8e-4],
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        showexponent='last',
        tickfont=dict(size=12),  
        range=[0, 8e4],
    ),
    yaxis2=dict(
        title=r"$\mathcal{\dot K}$",
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        showexponent='last',
        tickfont=dict(size=12),  
        overlaying='y',
        side='right',
        position=1,
        range=[0, 1.0e6],
    ),
    
    font=dict(
        family="Latin-Modern",
        size=12,
        color="Black"
    ),
    legend=dict(
        x=0.9,              # position from the left (0 to 1)
        y=0.58,              # position from the bottom (0 to 1)
        bgcolor='rgba(255, 255, 255, 0.5)',  # white with 80% opacity
        bordercolor='black',
        borderwidth=1,
        orientation='v',   
        xanchor='right',
        yanchor='top',
    ),
    width=400, 
    height=400,
    showlegend=False,
    template="plotly_white")
fig.show()
#fig.write_image("../output/plots/paper/overshoot_vs_time_unstable_t04_v2.pdf", width=400, height=280)

In [11]:
bbb

NameError: name 'bbb' is not defined

In [ ]:
file_paths = ['../output/clusters/paper/strain_rate_5.0e+03_safety_factor_4.0e-01_time1.0e-04_contact10/data.h5',
              '../output/clusters/paper/strain_rate_5.0e+03_safety_factor_4.0e-01_time1.0e-04_contact10adaptive/data.h5']
datasets = []
for file_path in file_paths:
    data = readH5Data(file_path)
    datasets.append(data)

In [ ]:
# Plot dissipated energy vs time
names = [r'$\text{Fixed } k^-$',
         r'$\text{Adaptive } k^-$']
dashes = ['solid', 'dot'] 
fig = go.Figure()
for i, data in enumerate(datasets):
    fig.add_trace(go.Scatter(x=data['time'], y=(data['edis']-data['edis'][0])/inj_energy_final, mode='lines', name=names[i], line=dict(color='grey', dash=dashes[i]), 
                             yaxis='y1', fill='tozeroy', fillcolor='rgba(211, 211, 211, 0.6)', showlegend=False))
    fig.add_trace(go.Scatter(x=data['time'], y=data['nb_fragments'], mode='lines', name=names[i], line=dict(color='black', dash=dashes[i]), yaxis='y2', showlegend=True))
fig.update_layout(
    title='Dissipated energy vs Time',
    xaxis_title='Time (s)',
    yaxis_title=r"$\mathcal{G}/\mathcal{E}_\text{inj}$",
    xaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        showexponent='last',
        tickfont=dict(size=12),
        range=[0, 1e-4],
        dtick=2e-5,
    ),
    yaxis=dict(
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        tickfont=dict(size=12),  
        range=[0, 0.6],
    ),
    yaxis2=dict(
        title=r"$N_\text{frag}$",
        showgrid=False,
        showline=True,
        linewidth=1,
        linecolor='black',
        mirror=True,
        zeroline=False,
        ticks='inside',
        exponentformat='power',
        tickfont=dict(size=12),  
        overlaying='y',
        side='right',
        position=1,
        range=[0, 800],
    ),

    font=dict(
        family="Latin-Modern",
        size=12,
        color="Black"
    ),
    legend=dict(
        x=0.97,              # position from the left (0 to 1)
        y=0.27,              # position from the bottom (0 to 1)
        bgcolor='rgba(255, 255, 255, 0.8)',  # white with 80% opacity
        bordercolor='black',
        borderwidth=1,
        orientation='v',
        xanchor='right',
        yanchor='top',
    ),
    width=400,
    height=400,
    showlegend=True,
    template="plotly_white")
fig.show()
#fig.write_image("../output/plots/paper/dissipated_energy_and_nfrag_vs_time_t04.pdf", width=400, height=400)